<a href="https://colab.research.google.com/github/josezerda/Trabajo_Final_DL_StyleTransfer/blob/main/TrabajoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cargamos las imagenes

In [1]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

--2024-02-26 13:14:57--  https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223725 (218K) [image/jpeg]
Saving to: ‘La_noche_estrellada1.jpg’

La_noche_estrellada 100%[===================>] 218.48K  --.-KB/s    in 0.09s   

2024-02-26 13:14:57 (2.26 MB/s) - ‘La_noche_estrellada1.jpg’ saved [223725/223725]

--2024-02-26 13:14:58--  https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153015 (149K) [ima

## Importamos librerias

In [30]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse
import tensorflow as tf

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path
tf.compat.v1.disable_eager_execution()

## Definimos las imagenes que vamos a utilizar, y el directorio de salida

In [31]:
base_image_path = Path("/content/775px-Neckarfront_Tübingen_Mai_2017.jpg")
style_reference_image_path = Path("/content/La_noche_estrellada1.jpg")
result_prefix = Path("/content/output")
iterations = 100

## 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

In [32]:
total_variation_weight = 0.1
style_weight = 10
content_weight = 1

In [33]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

### Respuesta:
Teniendo en cuenta que Style Transfer consiste en generar una imagen a partir de 2 imagenes (de 1 saca el estilo y de la otra el contenido). Esto se logra mediante la optimización de la loss function que tiene 3 componentes:
- total_variation_weight = Este valor impone una continuidad espacial local entre los píxeles de la imagen combinada lo que le da coherencia visual

- style_weight = Aca es donde aplicamos Deep Learning mediante el uso de una red CNN. Precisamente, consiste en una suma de distancias L2 entre las matrices de Gram de las representaciones de la imagen base y la imagen de referencia de estilo, extraídas de diferentes capas de una CNN (entrenado en ImageNet). La idea general es capturar información de color/textura a diferentes escalas espaciales (escalas bastante grandes –definidas por la profundidad de la capa considerada).

- content_weight = Este peso es la distancia L2 entre las características de la imagen base (extraída de una capa profunda) y las características de la imagen combinada, manteniendo la imagen generada lo suficientemente cerca de la original.

## 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

In [34]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

### Respuesta:
La funcion "preprocess_image", que tiene como argumento el path de una imagen, tiene como tarea realizar el preprocesamiento de una imagen a fin de que la misma pueda ser pasada a la CNN de 19 capas VGG19.

**img = np.expand_dims(img, axis=0):** A la imagen (img) que esta representada en el formato de un array de Numpy, se le agrega una nueva dimension al comienzo del mismo. Esto es necesario porque VGG19 espera un batch de imágenes como entrada, incluso si solo estás procesando una imagen. La nueva dimensión representa el tamaño del batch, que es 1 en este caso.

**img = vgg19.preprocess_input(img):** Aplica pasos de preprocesamiento específicos diseñados para el modelo VGG19. Estos pasos son:
- Resta los valores RGB medios del conjunto de datos ImageNet (con lo que se entreno VGG19) a cada píxel.
- Divide cada píxel por el valor de desviación estándar.
- Normaliza los valores de cada pixel [0,1].
- Conversión del espacio de color de RGB a BGR (si el modelo se entrenó en imágenes BGR).

## 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

In [35]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

### Respuesta:
La funcion "deprocess_image", revierte el procesamiento realizado por la funcion de la celda anterior "preprocess_image" para que la misma pueda ser utilizada por el modelo preentrenado VGG19. La funcion "deprocess_image" toma una imagen representada como un array Numpy y la transforma para que vuelva a ser interpretada por el ojo humano.


In [36]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [37]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [38]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [39]:

# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

Model loaded.


## 4) En la siguientes celdas:
- ¿Qué es la matriz de Gram?¿Para qué se usa?
- ¿Por qué se permutan las dimensiones de x?

In [40]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

### Respuesta:
La matriz de Gram se utiliza para captura el estilo de una imagen.
La matriz de Gram se calcula tomando los productos internos de las características extraídas de una capa de una red CNN (VGG19). Estas características son representaciones de alto nivel de la imagen original en el espacio de características de la red neuronal. Al calcular el producto interno de estas características, se obtiene información sobre las correlaciones entre las diferentes características.
En Neural Style Transfer, las matrices de Gram se calculan tanto para la imagen de estilo como para la imagen generada. El objetivo es generar una imagen que conserve el contenido de una imagen de contenido mientras adopta el estilo de una imagen de estilo. Al comparar las matrices de Gram de la imagen de estilo y la imagen generada en múltiples capas de CNN (VGG19), el algoritmo puede ajustar la imagen generada para que coincida con el estilo de la imagen de estilo. Esto generalmente se hace minimizando la diferencia (por ejemplo, usando un error cuadrático medio u otras funciones de loss) entre las matrices de Gram de la imagen de estilo y la imagen generada a traves de las diferentes capas.

En Neural Style Transfer, se permutan las dimensiones de x para calcular la matriz de Gram porque nos interesa calcular las correlaciones entre las características de estilo de una imagen, y la permutación facilita este cálculo.

Al permutar las dimensiones de x, transformamos el tensor de características en una matriz donde las filas representan los diferentes canales y las columnas representan la concatenación de las características espaciales. Esto nos permite calcular la matriz de Gram tomando el producto interno de estas características, lo que revela cómo se relacionan entre sí los diferentes canales de características.  Al permutar las dimensiones de x, podemos calcular eficazmente estas correlaciones y, por lo tanto, extraer información crucial sobre el estilo de la imagen de referencia. Esto es fundamental para lograr una transferencia de estilo efectiva en Neural Style Transfer.


## 5) Losses:
- Explicar qué mide cada una de las losses en las siguientes tres celdas.

In [41]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

### Respuesta:
La funcion "style_loss" cuantifica qué tan bien la imagen de salida coincide con las características de estilo de la imagen de referencia.

**Proposito de la funcion "style_loss"**
Al minimizar la funcion "style_loss", fomentamos que la imagen combinada coincida con las características de estilo (texturas, patrones) de la imagen de estilo. Las características de la imagen de contenido (capturadas por la funcion "content_loss") no varian.

**Argumentos de entrada:**
- style: Representa la imagen de estilo (generalmente una obra de arte)
- combination: Representa la imagen combinada (imagen contenido + imagen de estilo).

**Uso de la matriz Gram:**
La matriz de Gram captura las correlaciones entre los diferentes mapas de características (canales) en el tensor. Se utiliza para medir la similitud de estilo entre la imagen de estilo y la imagen combinada.

**Calculo de funcion de Style Loss**
La funcion de "style_loss" se calcula como la diferencia cuadrática media entre las matrices de Gram de la imagen de estilo (S) y la imagen combinada (C).
Se tiene en cuenta
- Numero de canales (generalmente 3 para  imagenes RGB )
- Numero total de pixels in the image (height x width).


In [42]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

### Respuesta:

La funcion "content_loss" cuantifica qué tan bien la imagen de salida coincide con las características de la imagen de contenido.

**Proposito de la funcion "content_loss"**

La función "content_loss" nos garantiza que la imagen generada conserve los detalles estructurales esenciales de la imagen del contenido al tiempo que incorpora el estilo deseado.

**Argumentos de entrada:**

- base: Representa la imagen original del contenido (feature map).
- combination: Representa la imagen generada (feature map).

**Calculo de funcion de Style Loss**

La funcion "content_loss" calcula la suma de las diferencias al cuadrado entre los elementos correspondientes de base y combinación. Cuantifica qué tan bien el contenido generado coincide con el contenido de referencia.
En Neural Style Transfer, minimizar la content loss ayuda a garantizar que la imagen generada capture las características esenciales de la imagen de contenido y al mismo tiempo permita variaciones artísticas.

In [43]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

### Respuesta:
**Proposito:**
La funcion "total_variation_loss" es usada en Neural Style Transfer y ayuda a crear imagenes visualmente coherentes imponiendo una continuidad espacial entre los pixeles adyacentes.

**Argumentos de entrada:**

x: Un tensor 4D que representa la imagen generada.

**Calculo de funcion de la Total Loss**

Esta funcion calcula la diferencia cuadrada entre pixels adyacentes a lo largo de ambos ejes (horizontal and vertical). Al minimizar la "total_variation_loss", procuramos que la imagen generada tenga transiciones más suaves entre los píxeles vecinos.


In [44]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

## 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

In [45]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

### Respuesta:

 Este código prepara una función que puede calcular tanto la función de loss como los gradientes de esa función de loss con respecto a la imagen combinada, lo que será útil para el proceso de optimización en el que se ajusta la imagen de combinación para que coincida con el estilo y el contenido deseados.

Cálculo de los gradientes:

"grads = K.gradients(loss, combination_image)": Esta línea utiliza la función K.gradients de Keras (o TensorFlow) para calcular los gradientes de la función de loss "loss" con respecto a la imagen de combinación "combination_image". Aquí, loss es la función de loss que se utiliza para medir la diferencia entre la imagen generada y la imagen de estilo, "combination_image" es la variable sobre la cual queremos calcular los gradientes, que es la imagen que estamos optimizando para que coincida con el estilo de la imagen de estilo y el contenido de la imagen de contenido.

Armado de las salidas:

"outputs = [loss]": Se crea una lista llamada outputs que contiene inicialmente solo la función de pérdida loss.
"if isinstance(grads, (list, tuple)): outputs += grads": Se verifica si grads es una lista o tupla (lo que implica que hay múltiples gradientes). Si es así, se agregan todos los gradientes a la lista outputs.
"else: outputs.append(grads)": Si grads no es una lista o tupla, se asume que es un solo gradiente y se agrega a la lista outputs.

Definición de la función de salida:

"f_outputs = K.function([combination_image], outputs)": Finalmente, se utiliza K.function para crear una función llamada "f_outputs" que toma la imagen de combinación como entrada y devuelve las salidas definidas anteriormente (que incluyen la función de pérdida y los gradientes, si existen) cuando se evalúa con la imagen de combinación.

In [46]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

### Respuesta:

Función: "eval_loss_and_grads"
Esta función se utiliza en el contexto de optimización de Neural Style Transfer.
Su objetivo es calcular tanto la funcion de loss del contenido (loss) como los gradientes asociados a una imagen de entrada x.

Parámetros:
x: Representa una imagen (generalmente generada por un modelo) que se desea evaluar.

Funcionalidad

x se redimensiona a una forma específica: (1, img_nrows, img_ncols, 3). Esto asegura que la entrada sea un tensor 4D con un solo ejemplo.
Se llama a la función "f_outputs" con x como argumento. Esta función devuelve una lista de valores (loss y gradientes).
Se extrae el valor de loss (loss_value) de la primera posición de la lista outs.
Si hay más elementos en la lista outs, se asume que el segundo elemento es el gradiente. En ese caso, se aplana y se convierte a tipo float64.
Si hay más de dos elementos en outs, se consideran como otros valores (por ejemplo, más gradientes) y se aplana y convierte en un arreglo de tipo float64.
Finalmente, la función devuelve tanto el valor de loss como los gradientes.
Uso en Neural Style Transfer:
Durante la optimización, se necesita evaluar la pérdida y los gradientes para ajustar la imagen generada. Los gradientes se utilizan para actualizar la imagen y lograr una mejor combinación de estilo y contenido.

En resumen, esta función toma una imagen, la preprocesa, llama a la función f_outputs para obtener los valores de loss y los gradientes, extrae los valores individuales y los devuelve ambos.

In [47]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

### Respuesta:

Clase: Evaluator
La clase Evaluator se utiliza en el contexto de optimizaciónen neural style transfer.
Su objetivo es calcular tanto la funcion de loss del contenido (loss) como los gradientes asociados a una imagen de entrada.

Métodos:
__init__(self): El método de inicialización (__init__) crea una instancia de la clase Evaluator. Inicializa dos atributos: loss_value y grads_values, ambos inicialmente como None.

loss(self, x): Este método calcula el valor de la funcion de loss (loss) para una imagen de entrada x. Utiliza la función eval_loss_and_grads(x) para obtener tanto la loss como los gradientes. Luego, almacena estos valores en los atributos de la instancia y devuelve el valor de la funcion de loss.

grads(self, x): Este método devuelve los gradientes previamente calculados. Antes de hacerlo, verifica que la funcion de loss ya haya sido calculada (es decir, self.loss_value no sea None). Luego, copia los gradientes almacenados y restablece los atributos de la instancia.

Uso en Neural Style Transfer:
Durante la optimización, se necesita evaluar la funcion de loss y los gradientes para ajustar la imagen generada.
La clase Evaluator encapsula esta funcionalidad, permitiendo calcular la funcion de loss y los gradientes de manera eficiente.
En resumen, la clase Evaluator es una parte importante del proceso de optimización en algoritmos como neural style transfer. Ayuda a mantener el estado de los valores de la funcion de loss y gradientes mientras se ajusta la imagen generada.

## ¿Qué hace la función fmin_l_bfgs_b?

La "función fmin_l_bfgs_b" es parte de la biblioteca SciPy y se utiliza para minimizar una función utilizando el algoritmo L-BFGS-B (Limited-memory Broyden-Fletcher-Goldfarb-Shanno with box constraints).

Objetivo:
Minimiza una función dada utilizando el algoritmo L-BFGS-B.
Este algoritmo es especialmente útil para problemas de optimización con restricciones de límites en las variables.

Parámetros:
func: La función que se desea minimizar.
x0: La estimación inicial de la solución.
fprime: El gradiente de la función (opcional).
bounds: Pares (mínimo, máximo) para cada elemento en x, definiendo los límites en esa dirección.
Otros parámetros como factr, pgtol, maxfun, maxiter, etc.

Salida:
x: La posición estimada del mínimo.
f: El valor de la función en el mínimo.
Información adicional en un diccionario.

En resumen, fmin_l_bfgs_b es una herramienta poderosa para resolver problemas de optimización con restricciones. Ayuda a encontrar los valores óptimos de las variables mientras cumple con las limitaciones impuestas.

## ¿En qué se diferencia con la implementación del paper?

Diferencias:

- El paper de Gatys propone un enfoque completamente diferente para combinar contenido y estilo en imágenes artísticas.
- En lugar de utilizar métodos de optimización como L-BFGS-B, el algoritmo de Gatys se basa en redes neuronales convolucionales (CNN) preentrenadas.
- El algoritmo de Gatys utiliza representaciones neuronales para separar y recombinar contenido y estilo de imágenes arbitrarias.
- A través de estas representaciones, logra crear imágenes artísticas de alta calidad que capturan tanto el contenido como el estilo deseado.

En resumen, mientras que fmin_l_bfgs_b es una herramienta general para optimización, el algoritmo de Gatys es específico para la transferencia de estilo artístico y utiliza redes neuronales para lograr resultados

## ¿Se puede utilizar alguna alternativa?


Sí, existen algunas alternativas a la función "fmin_l_bfgs_b" para la optimización en Neural Style Transfer:

1. Optimizadores Adam y SGD:
El optimizador Adam (Adaptive Moment Estimation) y el SGD (Stochastic Gradient Descent) son dos algoritmos populares para la optimización en redes neuronales.
Ambos ofrecen ventajas sobre fmin_l_bfgs_b:
Mayor velocidad de convergencia: Adam y SGD pueden converger más rápido que fmin_l_bfgs_b, especialmente en problemas con grandes conjuntos de datos.
Facilidad de implementación: Son más fáciles de implementar y configurar que fmin_l_bfgs_b.
Sin embargo, pueden tener algunos inconvenientes:
Menor precisión: Es posible que no alcancen la misma precisión que fmin_l_bfgs_b en algunos casos.
Sensibilidad a la tasa de aprendizaje: La elección de la tasa de aprendizaje puede ser crucial para el éxito de estos algoritmos.

2. Optimizadores de segundo orden:
Existen optimizadores de segundo orden como L-BFGS (Limited-memory Broyden–Fletcher–Goldfarb–Shanno) que pueden ofrecer mayor precisión que fmin_l_bfgs_b.
Sin embargo, son más costosos computacionalmente y pueden no ser adecuados para grandes conjuntos de datos.

3. Scipy’s "minimize" con otros métodos:
La biblioteca Scipy proporciona varios otros métodos de optimización además de L-BFGS-B.
Algoritmos como trust-constr, SLSQP, COBYLA.

4. Algoritmos Evolutivos:
Los algoritmos evolutivos, como el algoritmo de optimización de enjambre de partículas (PSO) o el algoritmo genético, también pueden ser alternativas interesantes.
Estos métodos se inspiran en la evolución biológica y pueden ser útiles para problemas de optimización más complejos.

5. Bibliotecas de optimización:
Existen bibliotecas de optimización como TensorFlow Optimizers y PyTorch Optimizers que ofrecen una amplia gama de algoritmos para la optimización.

La elección del mejor método depende de varios factores:
- Tamaño del conjunto de datos: Adam y SGD son más eficientes para grandes conjuntos de datos.
- Precisión requerida: Si se requiere la máxima precisión, fmin_l_bfgs_b o un optimizador de segundo orden puede ser mejor.
- Tiempo disponible: Si el tiempo es limitado, Adam o SGD pueden ser mejores opciones.


## 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [ ]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 12398301000.0
Image saved as /content/output/output_at_iteration_0.png
Iteration 0 completed in 504s
Start of iteration 1
Current loss value: 6042097000.0
Image saved as /content/output/output_at_iteration_1.png
Iteration 1 completed in 471s
Start of iteration 2
Current loss value: 4105960200.0
Image saved as /content/output/output_at_iteration_2.png
Iteration 2 completed in 476s
Start of iteration 3
Current loss value: 3186917600.0
Image saved as /content/output/output_at_iteration_3.png
Iteration 3 completed in 464s
Start of iteration 4
Current loss value: 2601488600.0
Image saved as /content/output/output_at_iteration_4.png
Iteration 4 completed in 466s
Start of iteration 5
Current loss value: 2250572300.0
Image saved as /content/output/output_at_iteration_5.png
Iteration 5 completed in 466s
Start of iteration 6
Current loss value: 2018009600.0
Image saved as /content/output/output_at_iteration_6.png
Iteration 6 completed in 462s
Start of ite